In [103]:
# # Installations for your venv
# %pip install numpy
# %pip install pandas
   # to split strings using regex


# Imports and Variables

In [ ]:
import os
import sys
import re
import pandas as pd
import numpy as np
import argparse
import sys

# import Emily's orTree module
import orTree



Parse command line

## Parse file - Using DataFrames

In [ ]:
def process_slots(data: str, columns: list, event_type: str, verbose = 1):
    """
    Processes a string of slot data and converts it into a pandas DataFrame.
    Args:
        data (str): A string containing slot data, with each slot separated by a newline.
        columns (list): A list of column names for the DataFrame.
        event_type (str): A character indicating either games 'G' or practices 'P'
        verbose (int, optional): Verbosity level. If set to 1, prints the processed DataFrame. Defaults to 1.
    Returns:
        pd.DataFrame: A DataFrame containing the processed slot data.
    """
    slots = []
    indices = []
    for slot in data.split('\n'):
        slot = slot.replace(' ','')
        if len(slot) > 0:
            # print(slot)
            day, time, max, min = slot.strip().split(',')
            # if not properties == [''] :
            slots.append({'Day': day,
                          'Start': time,
                          'Max': max,
                          'Min': min})
            indices.append(day + time)
    df = pd.DataFrame(slots, columns=columns, index=indices)
    df['Invalid_Assign'] = np.empty((len(df), 0)).tolist()
    df['Type'] = event_type
    if verbose:
        print(f'Processed {len(df)} {event_type} slots: \n {df}\n')
    return df


**Example of the Slots table**
|  Index  | Day     | Time    | Gamemin | Gamemax | Count   | Invalid_Assign                      | Type   |
|  -----  | ------- | ------- | ------- | ------- | ------- | ----------------------------------- | ------ |
| MO10:00 | MO      | 10:00   | 3       | 4       | 3       | [CMSAU17T2PRC01, CMSAU13T3DIV02]    | G      |

Example for verification:
1. Game max hard constraint

     ```slotindex[Count] > slotindex[Gamemax]```
2. Game min soft constraint

     ```slotindex[Count] > slotindex[Gamemin]```
3. Incompatible, Divs, Tiers hard constraints

     if most recent assignment is x, check if x is in slotindex[Invalid_Assign] ?

The following columns are filled during parsing and do not change:
- Index
- Day
- Time 
- Gamemin
- Gamemax

The following would need to be updated as assignments are made:
- Count: Increment count of appropriate slot for each assignment 
- Incompatible: Add incompatible pair 'partner' of assigned event (See intended lookup in #3).
     - We could also create a list of all events with a given 'tier' by filtering the games/practices dataframes.

In [106]:
def get_index(event: str):
    """
    Convert a string to desired index in DataFrame
    """
    return event.replace(' ', '')

In [ ]:
def process_games_practices(data, event_type, verbose = 1):
    """
    Processes game and practice data and converts it into a pandas DataFrame.
    Args:
        data (str): The raw data as a string, where each line represents a game or practice entry.
        columns (list): A list of column names for the resulting DataFrame.
        verbose (int, optional): Verbosity level. If set to 1, prints the processed DataFrame. Defaults to 1.
    Returns:
        pd.DataFrame: A DataFrame containing the processed game and practice data.
    """
    
    items = []
    indices = []
    strings = data.split('\n')
    for item in strings:
        index = get_index(item)
        # print(index)
        if len(index) > 0: # If not empty
            list_attributes = re.split(r'\s+', item.strip())
            if event_type == 'P' and len(list_attributes) == 6: # Normal practice
                # print(item)
                league, tier, _, divnum, ptype, pnum = list_attributes
            elif event_type == 'P' and len(list_attributes) == 4: # Practice used by all Divs
                # print(item)
                league, tier, ptype, pnum = list_attributes
                divt = 'DIV'
                divnum = ''
            elif event_type == 'G' and len(list_attributes) == 4: # Game
                # print(item)
                league, tier, _, divnum = list_attributes
                ptype = ''
                pnum = ''
            else: # Something is wrong
                raise ValueError(f"Invalid data format for event type {event_type}: {item}")
            # Dictionary of this item
            items.append({'League': league,
                          'Tier': tier,
                          'Div': divnum,
                          'Practice_Type': ptype,
                          'Num': pnum})
            indices.append(index)
    # DataFrame with items
    df = pd.DataFrame(items, index = indices)
    df['Type'] = event_type
    if verbose:
        print(f'Processed {len(df)} {event_type} items: \n {df}\n')
    return df


**Example of the Games Lookup Table**
|  Index           | League     | Tier    | Div     | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  --------------  | ---------- | ------- | ------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02  | CMSA       | U13T3   | DIV02   | TU9:30   | MO8:00      | 10         | [CMSAU19T3DIV01] | G    |          |


**Example of the Practices Lookup Table**
|  Index                | League     | Tier    | Div     | Type     | Num      | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  -------------------  | ---------- | ------- | ------- | -------- | -------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02OPN03  | CMSA       | U13T3   | DIV02   | OPN      | 03       | TU10:00  | TU17:00     | 10         | [CMSAU13T3DIV02] | P    |

**Inner Join creating Events Lookup Table**
|  Index                | League     | Tier    | Div     | Type     | Num      | Unwanted | Pref_slot   | Pref_value | Incompatible     | Type | Assigned |
|  -------------------  | ---------- | ------- | ------- | -------- | -------- | -------- | ----------- | ---------- | ---------------- | ---- | -------- |
|  CMSAU13T3DIV02OPN03  | CMSA       | U13T3   | DIV02   | OPN      | 03       | TU10:00  | TU17:00     | 10         | [CMSAU13T3DIV02] | P    | MO10:00
|  CMSAU13T3DIV02       | CMSA       | U13T3   | DIV02   |          |          | TU9:30   | MO8:00      | 5          | [CMSAU19T3DIV01] | G    |

Example Verification method:
1. Unwanted hard constraint

     ```if events['unwanted'] == events['Assigned']```
2. Preference soft constraint

     ```if assigned /= events['Pref_slot']``` then use ```events['Pref_value']``` to calculate penalty



-- Ideas:
To calculate the eval function, we can just inner join the Game_slots with the games dataframe and practice slots with the practices dataframe and vectorize values


List of strings of games and practices

In [108]:
# Strings Example
# games2 = split_data[3].split('\n')

# practice2 = split_data[4].split('\n')

def get_prop(event: str, feature):
    """
    Extracts a specific property from an event string based on the given feature.
    Parameters:
    event (str): The event string containing various properties separated by spaces or 'DIV'.
    feature (str): The feature to extract from the event string. 
                   Valid options are "League", "Tier", "Div", "Type", and "Num".
    Returns:
    str: The extracted property based on the specified feature.
    Raises:
    ValueError: If the feature is invalid or if the event string does not contain the required number of properties.
    """
    properties = event.strip().split(r'\s+| DIV')
    if feature == "League":
        return properties[0]
    elif feature == "Tier":
        return properties[1]
    elif feature == "Div":
        return properties[2]
    elif feature == "Type":
        if len(properties) >3:
            return properties[3]
        else: raise ValueError("Called Type on game or Type property is missing in the event data")
    elif feature == "Num":
        if len(properties) > 4:
            return properties[4]
        else: raise ValueError("Called Num on game or Type property is missing in the event data")
    else:
        raise ValueError("Invalid feature")

In [ ]:
def parse(input_data: str, verbose = 1):
    """
    Parses the input data and returns a dictionary containing the processed data.
    Args:
        input_data (str): The raw input data as a string.
    Returns:
        dict: A dictionary containing the processed data.
    """
    # Process game slots
    game_slots = process_slots(input_data[2], ['Day', 'Start', 'Max', 'Min', 'Invalid_Assign'], 'G')
        
    # Process practice slots
    practice_slots = process_slots(input_data[3], ['Day', 'Start', 'Max', 'Min', 'Invalid_Assign'], 'P')
    
    # Process games into a lookup table
    games = process_games_practices(input_data[4], 'G')

    # Process practices into a lookup table
    practices = process_games_practices(input_data[5], 'P')

    # Combine Practices and Games
    events = pd.concat([games, practices], axis=0)
    events = events.drop_duplicates()

    special_practices = []
    def special_detection(event):
        if event['League'] == 'CMSA' and event['Tier'] == 'U12T1':
            U12_practice = ['CMSA','U12T1S', '', '', '', 'P']
            special_practices.append(U12_practice)
        elif event['League'] == 'CMSA' and event['Tier'] == 'U13T1':
            U13_practice = ['CMSA','U13T1S', '', '', '', 'P']
            special_practices.append(U13_practice)

    events.apply(special_detection, axis=1)
    special_df = pd.DataFrame(special_practices, columns=['League', 'Tier', 'Div', 'Practice_Type', 'Num', 'Type'])
    special_df['LeagueTier'] = special_df['League'] + special_df['Tier']
    special_df = special_df.drop_duplicates(subset='LeagueTier', keep='first')
    special_df.set_index('LeagueTier', inplace=True)
    
    events = pd.concat([events, special_df], axis=0)

    # Prepare columns of empty lists
    events['Unwanted'] = np.empty((len(events), 0)).tolist()
    events['Incompatible'] = np.empty((len(events), 0)).tolist()
    events['Pair_with'] = np.empty((len(events), 0)).tolist()
    events['Preference'] = np.empty((len(events), 0)).tolist()
    events['Part_assign'] = '*'
    
    # Add all unwanted slots to the list in 'Unwanted' column for each event mentioned
    for unwanted in input_data[7].split('\n'):
        unwanted = unwanted.replace(' ','')
        if len(unwanted)>0:
            event, day, time = unwanted.strip().split(',')
            if not event in events.index:
                print(f'Unwanted entry error: {event} is not in table')
            else:
                event = get_index(event)
                events.at[event, 'Unwanted'].append(day + time)
                
    # Add all slot preferences and their value to the 'Preference' column
    preferences = []
    for pref in input_data[8].split('\n'):
        pref = pref.replace(' ','')
        if len(pref) > 0:
            # print(pref)
            day, time, index, value = pref.strip().split(',')
            if not index in events.index:
                print(f'Preference entry error: {index} is not in table')
            else:
                events.at[index, 'Preference'].append((day + time, value))

    # Add all incompatible pairs to the list in 'Incompatible' column
    for pair in input_data[9].split('\n'):
        string = pair.replace(' ', '')
        if len(string) > 0: # If not empty
            # print(string)
            event1, event2 = string.strip().split(',')
            if event1 in events.index and event2 in events.index:
                events.at[event1, 'Pair_with'].append(event2)
                events.at[event2, 'Pair_with'].append(event1)
            else:
                print(f'Pairs entry error: {event1} or {event2} is not in table')
                
    # Partial assignments used when f_trans selects branchs, before random choices
    partial_assignments = []
    for assign in input_data[10].split('\n'):
        string = assign.replace(' ', '')
        if len(string) > 0: # If not empty
            team, day, time = string.strip().split(',')
            partial_assignments.append((team, day+time))
            if team in events.index:
                events.at[team, 'Part_assign'] = day+time
            else:
                print(f'Part Assign entry error: {team} is not in table')

    events['Part_assign'] = events['Part_assign'].astype(str)

    special_practices = ['CMSAU12T1S', 'CMSAU13T1S']
    df = events[(events['Part_assign'] != "TU1800") & (events['Part_assign'] != "*")]
    print(df)
    for special in special_practices:
        if special in events.index:
            if special in df.index:
                print(f'Special Practice error: {special} is assigned to slot other than TU1800')
            else:
                events.at[special, 'Part_assign'] = 'TU1800'
    
    if verbose:
        print(f'Special practices: \n{special_df}\n')
        print(f'Not compatible: \n{events["Incompatible"]}\n')
        print(f'Unwanted slots: \n{events["Unwanted"]}\n')
        print(f'Preferences: \n{events["Preference"]}\n')
        print(f'Pairs: \n{events["Pair_with"]}\n')
        print(f'Partial Assignments: {partial_assignments}')
    
    events = events
    
    return game_slots, practice_slots, events
    

In [ ]:
class environment:
    def __init__(self, file_name = None, integers = None):
        """
        Initializes the class with a file name and a list of integers.
        Parameters:
        file_name (str, optional): The name of the file to be processed. If not provided, the user will be prompted to input it.
        integers (list of int, optional): A list of 8 integers representing weights and penalties. If not provided, the user will be prompted to input them.
        
        Attributes:
        file_name (str): The name of the file to be processed.
        integers (list of int): A list of 8 integers representing weights and penalties.
        w_minfilled (int): Weight for minimum filled.
        w_pref (int): Weight for preferences.
        w_pair (int): Weight for pairs.
        w_secdiff (int): Weight for section differences.
        pen_gamemin (int): Penalty for game minimum.
        pen_pracmin (int): Penalty for practice minimum.
        pen_notpaired (int): Penalty for not paired.
        pen_section (int): Penalty for section.
        game_slots (list): List of game slots parsed from the file.
        practice_slots (list): List of practice slots parsed from the file.
        events (list): List of events and their properties parsed from the file.
        
        Raises:
        ValueError: If any of the integers provided are negative.
        FileNotFoundError: If the specified file does not exist.
        """
    
        if file_name is None:
            # Prompt user for file name and integers if not set in code
            self.file_name = input('File')
            self.integers = input('Weights and Penalties:').split(' ')
            self.integers = list(map(int, self.integers))
            print(f'File chosen = {self.file_name}')
            print(f'Integer inputs = {self.integers}\n')
        else:
            self.file_name = file_name
            self.integers = integers
            
        # check if file exists
        if os.path.isfile(self.file_name):
            with open(self.file_name, "r") as inputfile:   # opens file     
                data = inputfile.read()          # starts reading file
                
                # splits the file based on the key words
                split_data = re.split(r"Name:|Game slots:|Practice slots:|Games:|Practices:|Not compatible:|Unwanted:|Preferences:|Pair:|Partial assignments:", data, flags=re.IGNORECASE)
                # print(split_data)
        else: 
            print("Unable to open file. Please try again.")
            
        # check if integers are positive
        if any(i < 0 for i in self.integers):
            print("Please enter 8 positive integers.")
        else:
            self.w_minfilled, self.w_pref, self.w_pair, self.w_secdiff, self.pen_gamemin, self.pen_pracmin, self.pen_notpaired, self.pen_section = self.integers
            
        self.game_slots, self.practice_slots, self.events = parse(split_data)
        
    def set_file(self, file_name):
        """
        Manually sets the file name.
        Args:
            file_name (str): The name of the file.
        """
        self.file_name = file_name
    
    def set_weights(self, integers):
        """
        Manually sets the weights and penalties.
        Args:
            integers (list of int): A list of integers representing weights and penalties.
        """
        self.integers = integers
        self.w_minfilled, self.w_pref, self.w_pair, self.w_secdiff, self.pen_gamemin, self.pen_pracmin, self.pen_notpaired, self.pen_section = integers
      
    def get_events(self):
        """
        Returns the DataFrame containing the processed game and practice data.
        Returns:
            pd.DataFrame: A DataFrame containing the processed game and practice data.
        """
        return self.events
    
    def get_game_slots(self):
        """
        Returns the DataFrame containing the processed game slots.
        Returns:
            pd.DataFrame: A DataFrame containing the processed game slots.
        """
        return self.game_slots
    
    def get_practice_slots(self):
        """
        Returns the DataFrame containing the processed practice slots.
        Returns:
            pd.DataFrame: A DataFrame containing the processed practice slots.
        """
        return self.practice_slots
    
    def get_weights(self):
        """
        Returns the weights and penalties as a tuple.
        Returns:
            tuple: A tuple containing the weights and penalties.
        """
        return (self.w_minfilled, self.w_pref, self.w_pair, self.w_secdiff, self.pen_gamemin, self.pen_pracmin, self.pen_notpaired, self.pen_section)
    
    def get_penalties(self):
        """
        Returns the penalties as a tuple.
        Returns:
            tuple: A tuple containing the penalties.
        """
        return (self.pen_gamemin, self.pen_pracmin, self.pen_notpaired, self.pen_section)

In [110]:
env = environment()
events = env.get_events()
game_slots = env.get_game_slots()
practice_slots = env.get_practice_slots()



print(events)

# print(game_slots['Min'])


File chosen = Jamie.txt
Integer inputs = [1, 2, 1, 2, 5, 5, 5, 5]

Processed 3 G slots: 
        Day Start Max Min Invalid_Assign Type
MO8:00  MO  8:00   3   2             []    G
MO9:00  MO  9:00   3   2             []    G
TU9:30  TU  9:30   2   1             []    G

Processed 3 P slots: 
         Day  Start Max Min Invalid_Assign Type
MO8:00   MO   8:00   4   2             []    P
TU10:00  TU  10:00   2   1             []    P
FR10:00  FR  10:00   2   1             []    P

Processed 4 G items: 
                League   Tier Div Practice_Type Num Type
CMSAU13T1DIV01   CMSA  U13T1  01                      G
CMSAU13T3DIV02   CMSA  U13T3  02                      G
CUSAO18DIV01     CUSA    O18  01                      G
CMSAU17T1DIV01   CMSA  U17T1  01                      G

Processed 5 P items: 
                     League   Tier Div Practice_Type Num Type
CMSAU13T1DIV01PRC01   CMSA  U13T1  01           PRC  01    P
CMSAU13T1DIV01OPN02   CMSA  U13T1  01           OPN  02    P
CMSAU13

Tracker for games and practices

In [ ]:
# Class of schedule
debug_pair_with = True
test_stuff_debug = True


class Schedule:
    
    def __init__(self):
        self.eval = None
        self.event_list = events.index.tolist()
        self.assignments = events['Part_assign']
        self.assigned = []
        
    def get_Starting(self):
        # print(f"{self.assignments = }")
        return self.assignments.to_list()
        
    def set_Eval(self):
        total_penalty = 0
        total_df = events.copy()

        total_df['Assigned'] = self.assignments['Slot']
        total_df['Assigned'] = total_df['Assigned'].apply(lambda x: '' if isinstance(x, list) and not x else x)
        df = pd.merge(total_df, game_slots, how = 'left', left_on = 'Assigned', right_index = True)
        df = pd.merge(df, practice_slots, how = 'left', left_on = 'Assigned', right_index = True)    
        
        # print(df['Pref'])
        filterRows = df[df['Assigned'] != ""]

        game_slot_dic = dict()
        prac_slot_dic = dict()

        # keeping track of how many games are in each game slot for the game_min penalty
        for slot in env.game_slots.index:
            game_slot_dic[slot] = 0

        # keeping track of how many practices are in each practice_slot for the practice_min penalty
        for slot in env.practice_slots.index:
            prac_slot_dic[slot] = 0

        for row in filterRows.index:

            game_slot_dic[df['Assigned'][row]] += 1

            if(debug_pair_with):

                events_non_empty = events[events['Pair_with'].apply(lambda x: not isinstance(x, list) or len(x) > 0)]
                pair_with_dict = events_non_empty['Pair_with'].to_dict()
            # this is doing the work for the pair_with penalty    
            if pair_with_dict.get(row) is not None:
                games_to_check_for_row = pair_with_dict.get(row)
                if(test_stuff_debug):
                    print(f"{games_to_check_for_row = }")

                # check that the games and the row have the same slot 
                # todo one issuse if we have a game and a practice that are in a pair that could be annoying to check for 
                for game in games_to_check_for_row:
                    if(game in filterRows.index):
                        
                        # we have the game and we have the row check if game and row are the same or not
                        var = df['Assigned'][row]
                        v2 = df['Assigned'][game]
                        
                        if(var != v2):
                            cal = env.pen_notpaired * env.w_pair
                            total_penalty += cal / 2
                            

                    else:
                        print("game not in index")

            # end game pair stuff
            
            # now we gotta do the fit pref
            # we need to get the preferred values for the games and practices
            # if the games and practices are not in their preferred slot, we add a penalty
            events_pref_non_empty = events[events['Preference'].apply(lambda x: not isinstance(x, list) or len(x) > 0)]
            pref_with_dict = events_pref_non_empty['Preference'].to_dict()

            if pref_with_dict.get(row) is not None:
                slots_to_check = pref_with_dict.get(row)
 
                # check that the games and the row have the same slot 
                # todo one issuse if we have a game and a practice that are in a pair that could be annoying to check for 
                for slot in slots_to_check:
                    if(slot in filterRows.index):
                        
                        # we have the game and we have the row check if game and row are the same or not
                        var = df['Assigned'][row]
                        v2 = df['Assigned'][slot]
                        
                        if(var != v2):
                            pen = env.w_pref * df['Pref_value'][slot]
                            total_penalty += pen
                            
                
        # now lets figure out the slots and how many can go in each spot

        # each game has a slot assigned lets assume a complete schedule
        # we can make a dictionary of all the slots and then maybe fill in how many games that they have
        # then we can apply the penalty at the end if they don't have the minimun

        # to dict 
        # need a for loop here to go through each entry in the dict

        #game min section
        def game_min_penalty():
            penalty_game_min_total = 0
            current_game_penalty = 0
            env.pen_gamemin
            env.w_minfilled
            game_slots_min = game_slots['Min'].to_dict()
            for slot in game_slot_dic:
                current_game_penalty = env.pen_gamemin * env.w_minfilled * max(0, int(game_slots_min[slot]) - game_slot_dic[slot] )   
                penalty_game_min_total += current_game_penalty
            print(f"{penalty_game_min_total = }")

            return penalty_game_min_total

        total_penalty = game_min_penalty() + total_penalty
            
        
        

        def prac_min_penalty():
            penalty_prac_min_total = 0
            current_prac_penalty = 0
            env.pen_pracmin
            env.w_minfilled
            practice_slots_min = practice_slots['Min'].to_dict()
            for slot in prac_slot_dic:
                current_prac_penalty = env.pen_gamemin * env.w_minfilled * max(0, int(practice_slots_min[slot]) - prac_slot_dic[slot] )   
                penalty_prac_min_total += current_prac_penalty
            print(f"{penalty_prac_min_total = }")
            
            return penalty_prac_min_total

        total_penalty = prac_min_penalty() + total_penalty
        
            
        
        
        if(test_stuff_debug):
            print(f"{total_penalty = }")

        self.eval = total_penalty
    
    def assign(self, slots):
        self.assignments['Slot'] = slots
        return self.eval
    

sched = Schedule()
# print(sched.get_Starting())
sched.assign(["MO8:00","MO8:00","MO9:00","MO8:00","MO8:00","MO8:00","MO8:00","MO8:00","MO8:00","MO8:00"])
sched.set_Eval()


games_to_check_for_row = ['CMSAU17T1DIV01']
games_to_check_for_row = ['CUSAO18DIV01']
penalty_game_min_total = 10
penalty_prac_min_total = 20
total_penalty = 35.0


In [116]:
sched = Schedule()
print(sched.get_Starting())
# sched.set_Eval()

['*', '*', 'MO8:00', '*', '*', '*', '*', 'FR10:00', '*', 'TU1800']
